In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [67]:
df_routes=pd.read_csv('routes.csv')
df_incidents=pd.read_csv('incidents.csv')



In [68]:
df_routes.head(2)


,Route ID,Origin Port,Destination Port,Orgin Port Country,Destination Port Country,No Of Stops Between,Stops Names,"Stop Points (Orgin,[BetwwenStops],End]",Distance,Average Transit Time (days),Travel Direction,Vessel Type,No Of Travels
0,R001,Tanger Med,Rotterdam,Morocco,Netherlands,NaN,"Algeciras,Barcelona, Antwerp","[44.811855,-0.5906289]-[36.1311725,-5.4473991]...",600.0,18.0,W,Bulk carrier,15.0
1,R002,Gioia Tauro,Hamburg,Italy,Germany,NaN,"Gioia Tauro,Valencia,Barcelona,Marseille,Rotte...","[38.424938, 15.899940]-[40.7127281,-74.0060152...",2468.0,29.0,E,Tank,1500.0


In [69]:
df_incidents.head(2)

,Incidents,RouteID,IncidentType,Severity,Location,IncidentDate,Description,ResponseTime,DamageCost,InsuranceClaim
0,INC001,R001,Piracy,Medium,NaN,23-10-2022,NaN,2 hours,1000000.0,Yes
1,INC002,R001,Accident,Medium,NaN,26-01-2023,Offshore Oil Rig Mishaps,4 hours,500000.0,No


In [70]:
# Convert 'date' column to datetime with the correct format
df_incidents['IncidentDate'] = pd.to_datetime(df_incidents['IncidentDate'], format='%d-%m-%Y')

In [71]:
# Current date
current_date = datetime.now()

# Function to get the date 5 months ago
def date_n_months_ago(current_date, n):
    return current_date - timedelta(days=n*30)  # Approximate 5 months as 150 days

five_months_ago = date_n_months_ago(current_date, 5)

In [72]:
# Adding columns to routes dataframe
df_routes['total_incidents_count'] = 0
df_routes['high_incidents'] = 0
df_routes['high_last5Month_incidents'] = 0
df_routes['medium_incidents'] = 0
df_routes['medium_last5Month_incidents'] = 0
df_routes['low_incidents'] = 0
df_routes['low_last5Month_incidents'] = 0


In [73]:


for index,row in df_routes.iterrows():
    #total incidents and count
    route_incidents=df_incidents[df_incidents['RouteID']==row['Route ID']]   
    incidents_count=route_incidents.shape[0]
    df_routes.loc[index, 'total_incidents_count']=incidents_count
    
    #High incidents and count
    high_incidents = route_incidents[route_incidents['Severity'] == 'High']
    high_count=high_incidents.shape[0]
    high_last5Month_count=high_incidents[high_incidents['IncidentDate']>five_months_ago].shape[0]

    df_routes.loc[index, 'high_incidents']=(high_count-high_last5Month_count)
    df_routes.loc[index, 'high_last5Month_incidents']=high_last5Month_count

    
    
    #Medium incidents and count
    medium_incidents = route_incidents[route_incidents['Severity'] == 'Medium']
    medium_count=medium_incidents.shape[0]
    medium_last5Month_count=medium_incidents[medium_incidents['IncidentDate']>five_months_ago].shape[0]
    df_routes.loc[index, 'medium_incidents'] = (medium_count-medium_last5Month_count)
    df_routes.loc[index, 'medium_last5Month_incidents'] = medium_last5Month_count
    
        #Medium incidents and count
    low_incidents = route_incidents[route_incidents['Severity'] == 'Low']
    low_count=low_incidents.shape[0]
    low_last5Month_count=low_incidents[low_incidents['IncidentDate']>five_months_ago].shape[0]
    df_routes.loc[index, 'low_incidents'] = (low_count-low_last5Month_count)
    df_routes.loc[index, 'low_last5Month_incidents'] =low_last5Month_count
    


    
    


In [74]:
df_routes.head(2)

,Route ID,Origin Port,Destination Port,Orgin Port Country,Destination Port Country,No Of Stops Between,Stops Names,"Stop Points (Orgin,[BetwwenStops],End]",Distance,Average Transit Time (days),Travel Direction,Vessel Type,No Of Travels,total_incidents_count,high_incidents,high_last5Month_incidents,medium_incidents,medium_last5Month_incidents,low_incidents,low_last5Month_incidents
0,R001,Tanger Med,Rotterdam,Morocco,Netherlands,NaN,"Algeciras,Barcelona, Antwerp","[44.811855,-0.5906289]-[36.1311725,-5.4473991]...",600.0,18.0,W,Bulk carrier,15.0,5,2,0,3,0,0,0
1,R002,Gioia Tauro,Hamburg,Italy,Germany,NaN,"Gioia Tauro,Valencia,Barcelona,Marseille,Rotte...","[38.424938, 15.899940]-[40.7127281,-74.0060152...",2468.0,29.0,E,Tank,1500.0,7,2,0,3,0,2,0


In [75]:
# Adding columns to routes dataframe
df_routes['risk_score'] = 0

In [76]:
# df_routes['total_incidents_count'] = 0
# df_routes['high_incidents'] = 0
# df_routes['high_last5Month_incidents'] = 0
# df_routes['medium_incidents'] = 0
# df_routes['medium_last5Month_incidents'] = 0
# df_routes['low_incidents'] = 0
# df_routes['low_last5Month_incidents'] = 0

for index,row in df_routes.iterrows():
    total_travel_count=row['No Of Travels']
    total_incidents_count=row['total_incidents_count'] #200
    high_incidents=row['high_incidents'] #*3
    high_last5Month_incidents=row['high_last5Month_incidents'] #*5
    
    medium_incidents=row['medium_incidents'] #*2
    medium_last5Month_incidents=row['medium_last5Month_incidents'] #*3
    
    low_incidents=row['low_incidents'] #*1
    low_last5Month_incidents=row['low_last5Month_incidents'] #*2
    
    risk_value=((high_incidents*3)+(high_last5Month_incidents*5)+\
        (medium_incidents*2)+(medium_last5Month_incidents*3)+\
            (low_incidents*1)+(low_last5Month_incidents*2))/total_travel_count
        
    risk_value=risk_value*100;
    
    if risk_value>100:
        risk_value=98
    
    df_routes.loc[index,'risk_score']=risk_value
    
    

C:\Users\ponsi\AppData\Local\Temp\ipykernel_11636\1482049009.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9333333333333335' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_routes.loc[index,'risk_score']=risk_value


In [77]:
df_routes

,Route ID,Origin Port,Destination Port,Orgin Port Country,Destination Port Country,No Of Stops Between,Stops Names,"Stop Points (Orgin,[BetwwenStops],End]",Distance,Average Transit Time (days),...,Vessel Type,No Of Travels,total_incidents_count,high_incidents,high_last5Month_incidents,medium_incidents,medium_last5Month_incidents,low_incidents,low_last5Month_incidents,risk_score
0,R001,Tanger Med,Rotterdam,Morocco,Netherlands,NaN,"Algeciras,Barcelona, Antwerp","[44.811855,-0.5906289]-[36.1311725,-5.4473991]...",600.0,18.0,...,Bulk carrier,15.0,5,2,0,3,0,0,0,80.000000
1,R002,Gioia Tauro,Hamburg,Italy,Germany,NaN,"Gioia Tauro,Valencia,Barcelona,Marseille,Rotte...","[38.424938, 15.899940]-[40.7127281,-74.0060152...",2468.0,29.0,...,Tank,1500.0,7,2,0,3,0,2,0,0.933333
2,R003,Piraeus,Hamburg,Greece,Belgium,NaN,NaN,"[37.942986, 23.646982]-[51.219448, 4.402464]",3457.0,39.0,...,Container Shipo,789.0,9,3,2,2,0,2,0,3.168568
3,R004,Alexandria,Bremerhaven,Egypt,Germany,NaN,NaN,"[31.1991806,29.8951716]-[53.5505392,8.5851945]",4500.0,15.0,...,Reefer ship,1467.0,9,2,0,4,1,1,1,1.363327
4,R005,Port Sudan,Le Havre,Sudan,France,NaN,NaN,"[19.619217,37.211709]-[49.4938975,0.1079732]",200.0,27.0,...,Project cargo ship,938.0,7,2,0,5,0,0,0,1.705757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
